This demonstration is based off the companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). This notebook was generated for TensorFlow 2.6, but is current up to 2.12.0.

In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC

print(tf.__version__)

2023-07-31 20:28:47.341424: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-31 20:28:47.391896: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.0


## Demo Part I - Add TensorBoard to a model 
### The IMDB dataset
### ML problem: Binary classification, positive or negative reviews?

**Loading the IMDB dataset**

In [2]:
from tensorflow.keras.datasets import imdb

# Load training and test data from imdb dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

# Values of text from this dataset integer encoded; this first sample
# train_data[0]

# The label for the first sample (1 or 0)
# train_labels[0]

# The number of trainign samples 1000 total
# max([max(sequence) for sequence in train_data])

**Decoding reviews back to text**

In [3]:
# If curious about the decoded text, set DECODE to True and run.
# Only the first sample is decoded.
DECODE = True
if DECODE:
        
    word_index = imdb.get_word_index()
    reverse_word_index = dict(
        [(value, key) for (key, value) in word_index.items()])
    decoded_review = " ".join(
        [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

    # See the decoded version of the encoded words from the imdb dataset
    print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

### Preparing the data

**Encoding the integer sequences via multi-hot encoding**

In [4]:

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results
# create train and test datasets    
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

# See multi-hot encoded vectorrized sequence
# x_train[0]


In [5]:
# Cast as floats for training
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

# Create train/validation split
x_val = x_train[:10000]
y_val = y_train[:10000]
partial_x_train = x_train[10000:]
partial_y_train = y_train[10000:]

### Building your model using TensorBoard

In [6]:
# setup logging scheme
MODEL = "imdb-model-{}".format(int(time.time()))

# initialize callback (create logs directory first or model.fit will fail)
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# windows?
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

print(MODEL + " <- this will be your TensorBoard log file name.")

imdb-model-1690835335 <- this will be your TensorBoard log file name.


In [7]:
# initialize a sequential fully connected network to address the ML problem
model = keras.Sequential([
    layers.Dense(16, activation="relu", name="L1_16_Dense_RELU"),
    layers.Dense(16, activation="relu", name="L2_16_Dense_RELU"),
    layers.Dense(1, activation="sigmoid", name="L3_1_Dense_SIG")
])

# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# Fit your model, including TensorBoard in the callback list
model.fit(partial_x_train,
          partial_y_train, 
          validation_data=(x_val, y_val),
          epochs=20, 
          batch_size=512, 
          callbacks=[tbcallback])
            

2023-07-31 20:28:56.492025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30925 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:b3:00.0, compute capability: 7.0


Epoch 1/20


2023-07-31 20:29:02.107471: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fbd10017a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-31 20:29:02.107546: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-07-31 20:29:02.117429: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-31 20:29:02.331520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-31 20:29:02.489866: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


30/30 [==============================] - 7s 66ms/step - loss: 0.5591 - accuracy: 0.7641 - val_loss: 0.4371 - val_accuracy: 0.8529
Epoch 2/20
30/30 [==============================] - 1s 22ms/step - loss: 0.3616 - accuracy: 0.8878 - val_loss: 0.3506 - val_accuracy: 0.8715
Epoch 3/20
30/30 [==============================] - 1s 27ms/step - loss: 0.2721 - accuracy: 0.9131 - val_loss: 0.2955 - val_accuracy: 0.8853
Epoch 4/20
30/30 [==============================] - 1s 23ms/step - loss: 0.2187 - accuracy: 0.9290 - val_loss: 0.2917 - val_accuracy: 0.8843
Epoch 5/20
30/30 [==============================] - 1s 23ms/step - loss: 0.1804 - accuracy: 0.9420 - val_loss: 0.2819 - val_accuracy: 0.8826
Epoch 6/20
30/30 [==============================] - 1s 21ms/step - loss: 0.1546 - accuracy: 0.9503 - val_loss: 0.2832 - val_accuracy: 0.8843
Epoch 7/20
30/30 [==============================] - 1s 30ms/step - loss: 0.1330 - accuracy: 0.9568 - val_loss: 0.3358 - val_accuracy: 0.8703
Epoch 8/20
30/30 [======

### Review TensorBoard output to evaluate how model performance.
#### STOP and run all cells above and return to slides.

In [8]:
# In console, from directory containing the "logs" directory.
# tensorboard --logdir logs

### Refit model with adjusted parameters and all training data.

In [9]:
# Make any adjustments for your final model
model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# added in a Precision Recall curve metric
metrics=[tf.keras.metrics.Accuracy(name='accuracy'),
        tf.keras.metrics.AUC(name='prc', curve='PR')]


# Compile the network using your optmizier, loss, and metrics
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=metrics)

# Update logging scheme
MODEL = "imdb-final-model-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
# tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

# Refit your final model
model.fit(x_train,                              # No longer partial_x_train
          y_train,                              # No longer partial_y_train
          epochs=5,                             # Changed to 5 epochs 
          batch_size=512, 
          callbacks=[tbcallback])
            

Epoch 1/5
49/49 [==============================] - 5s 25ms/step - loss: 0.4717 - accuracy: 0.0000e+00 - prc: 0.8974
Epoch 2/5
49/49 [==============================] - 1s 14ms/step - loss: 0.2828 - accuracy: 0.0000e+00 - prc: 0.9602
Epoch 3/5
49/49 [==============================] - 1s 14ms/step - loss: 0.2196 - accuracy: 4.0000e-05 - prc: 0.9731
Epoch 4/5
49/49 [==============================] - 1s 14ms/step - loss: 0.1868 - accuracy: 4.0000e-05 - prc: 0.9799
Epoch 5/5
49/49 [==============================] - 1s 15ms/step - loss: 0.1643 - accuracy: 4.0000e-05 - prc: 0.9841


In [10]:

# Check how accurate the model is at predicting unseen observations
results = model.evaluate(x_test, y_test)

# A tuple containing loss, accuracy
results

782/782 [==============================] - 4s 4ms/step - loss: 0.2930 - accuracy: 4.0000e-05 - prc: 0.9443


[0.2929645776748657, 3.9999998989515007e-05, 0.9442622661590576]

### Using a trained model to generate predictions on new data

In [11]:
model.predict(x_test)

782/782 [==============================] - 2s 3ms/step


array([[0.17112797],
       [0.9997923 ],
       [0.81388503],
       ...,
       [0.10509251],
       [0.06741008],
       [0.68772835]], dtype=float32)

# Part 2: Comparing multiple models

## Classifying newswires: A multiclass classification example

### The Reuters dataset

In [12]:
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import reuters

**Loading the Reuters dataset**

In [13]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000)

#len(train_data)
#len(test_data)
#train_data[10]

**Decoding newswires back to text**

In [14]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in
    train_data[0]])

#train_labels[10]

### Preparing the data

**Encoding the input and labels & Splitting data**

In [15]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

#Encode input
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#Encode labels
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

# import 1-hot encoding library
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

#Set aside validation data
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]

### Building your model

**Model definition**

In [16]:
# Create two models, the only difference is hidden layer activations

model1 = keras.Sequential([
    layers.Dense(64, activation="tanh"),
    layers.Dense(64, activation="tanh"),
    layers.Dense(46, activation="softmax")
])

model2 = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])

**Compiling the model**

In [17]:
model1.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model2.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

**Training the model**

In [18]:
# setup logging scheme for model1
MODEL = "mod1-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history1 = model1.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

MODEL = "mod2-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

history2 = model2.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[tbcallback])

Epoch 1/20
16/16 [==============================] - 6s 80ms/step - loss: 2.4180 - accuracy: 0.4872 - val_loss: 1.7011 - val_accuracy: 0.6300
Epoch 2/20
16/16 [==============================] - 1s 39ms/step - loss: 1.4533 - accuracy: 0.6849 - val_loss: 1.3085 - val_accuracy: 0.7040
Epoch 3/20
16/16 [==============================] - 1s 37ms/step - loss: 1.1158 - accuracy: 0.7711 - val_loss: 1.1186 - val_accuracy: 0.7620
Epoch 4/20
16/16 [==============================] - 1s 48ms/step - loss: 0.8955 - accuracy: 0.8202 - val_loss: 1.0007 - val_accuracy: 0.7960
Epoch 5/20
16/16 [==============================] - 1s 40ms/step - loss: 0.7320 - accuracy: 0.8544 - val_loss: 0.9253 - val_accuracy: 0.8020
Epoch 6/20
16/16 [==============================] - 1s 34ms/step - loss: 0.6015 - accuracy: 0.8857 - val_loss: 0.8696 - val_accuracy: 0.8230
Epoch 7/20
16/16 [==============================] - 1s 42ms/step - loss: 0.4984 - accuracy: 0.9062 - val_loss: 0.8357 - val_accuracy: 0.8270
Epoch 8/20
16

### STOP and return to slide to discuss.

**Retraining a model from scratch**

In [26]:
MODEL = "final-reuters-{}".format(int(time.time()))
tbcallback = TensorBoard(log_dir = "logs/{}".format(MODEL), histogram_freq=1)
#tbcallback = TensorBoard(log_dir = "logs\{}".format(MODEL), histogram_freq=1)

model = keras.Sequential([
  layers.Dense(64, activation="tanh"),
  layers.Dense(64, activation="tanh"),
  layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.fit(x_train,
          y_train,
          epochs=9,
          batch_size=512,
          callbacks=[tbcallback])
          
results = model.evaluate(x_test, y_test)


Epoch 1/9
18/18 [==============================] - 2s 42ms/step - loss: 2.2777 - accuracy: 0.5428
Epoch 2/9
18/18 [==============================] - 0s 28ms/step - loss: 1.3639 - accuracy: 0.7043
Epoch 3/9
18/18 [==============================] - 0s 23ms/step - loss: 1.0436 - accuracy: 0.7823
Epoch 4/9
18/18 [==============================] - 0s 28ms/step - loss: 0.8299 - accuracy: 0.8327
Epoch 5/9
18/18 [==============================] - 0s 23ms/step - loss: 0.6696 - accuracy: 0.8686
Epoch 6/9
18/18 [==============================] - 1s 31ms/step - loss: 0.5414 - accuracy: 0.8942
Epoch 7/9
18/18 [==============================] - 1s 29ms/step - loss: 0.4446 - accuracy: 0.9135
Epoch 8/9
18/18 [==============================] - 1s 30ms/step - loss: 0.3677 - accuracy: 0.9290
Epoch 9/9
71/71 [==============================] - 0s 4ms/step - loss: 0.8688 - accuracy: 0.8028


In [20]:
results

[0.9200968742370605, 0.800979495048523]

In [21]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
hits_array.mean()

0.17764915405164738

### Generating predictions on new data

In [22]:
predictions = model.predict(x_test)

71/71 [==============================] - 0s 2ms/step


In [23]:
predictions[0].shape

(46,)

In [24]:
np.sum(predictions[0])

1.0

In [25]:
np.argmax(predictions[0])

3